In [1]:
import pandas as pd
import re
from pathlib import Path
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import pandas as pd
import re
from pathlib import Path
from nltk.tokenize import word_tokenize
import nltk
import grpc
import zemberek_grpc.morphology_pb2 as z_morphology
import zemberek_grpc.morphology_pb2_grpc as z_morphology_g
channel = grpc.insecure_channel('localhost:6789')
morphology_stub = z_morphology_g.MorphologyServiceStub(channel)
# Özel Türkçe stopwords listesi (NLTK'ya ek olarak)
extended_turkish_stopwords = {
    'a', 'acaba', 'acep', 'adamakıllı', 'adeta', 'ait', 'altmış', 'altı',
    'ama', 'amma', 'anca', 'ancak', 'arada', 'artık', 'aslında', 'aynen', 'ayrıca',
    'az', 'açıkça', 'açıkçası', 'bana', 'bari', 'bazen', 'bazı', 'başkası',
     'belki', 'ben', 'benden', 'beni', 'benim', 'beri', 'beriki', 'beş',
     'bilcümle', 'bile', 'bin', 'binaen', 'binaenaleyh', 'bir', 'biraz',
    'birazdan', 'birbiri', 'birden', 'birdenbire', 'biri', 'birice', 'birileri',
    'birisi', 'birkaç', 'birkaçı', 'birkez', 'birlikte', 'birçok', 'birçoğu',
    'bir şey', 'bir şeyi', 'birşey', 'birşeyi', 'bitevi', 'biteviye',
    'bittabi', 'biz', 'bizatihi', 'bizce', 'bizcileyin', 'bizden', 'bize', 'bizi',
    'bizim', 'bizimki', 'bizzat', 'boşuna', 'bu', 'buna', 'bunda', 'bundan',
    'bunlar', 'bunları', 'bunların', 'bunu', 'bunun', 'buracıkta', 'burada',
    'buradan', 'burası', 'böyle', 'böylece', 'böylecene', 'böylelikle',
    'böylemesine', 'böylesine', 'büsbütün', 'bütün', 'cuk', 'cümlesi', 'da',
    'daha', 'dahi', 'dahil', 'dahilen', 'daima', 'dair', 'dayanarak', 'de', 'defa',
    'dek', 'demin', 'demincek', 'deminden', 'denli', 'derakap', 'derhal', 'derken',
     'değil', 'değin', 'diye', 'diğer', 'diğeri', 'doksan', 'dokuz',
    'dolayı', 'dolayısıyla', 'doğru', 'dört', 'edecek', 'eden', 'ederek', 'edilecek',
    'ediliyor', 'edilmesi', 'ediyor', 'elbet', 'elbette', 'elli', 'emme', 'en',
    'enikonu', 'epey', 'epeyce', 'epeyi', 'esasen', 'esnasında', 'etmesi', 'etraflı',
    'etraflıca', 'etti', 'ettiği', 'ettiğini', 'evleviyetle', 'evvel', 'evvela',
    'evvelce', 'evvelden', 'evvelemirde', 'evveli', 'eđer', 'eğer', 'fakat',
    'filanca', 'gah', 'gayet', 'gayetle', 'gayri', 'gayrı', 'gelgelelim', 'gene',
    'gerek', 'gerçi', 'geçende', 'geçenlerde', 'gibi', 'gibilerden', 'gibisinden',
    'gine', 'göre', 'gırla', 'hakeza', 'halbuki', 'halen', 'halihazırda', 'haliyle',
    'handiyse', 'hangi', 'hangisi', 'hani', 'hariç', 'hasebiyle', 'hasılı', 'hatta',
    'hele', 'hem', 'henüz', 'hep', 'hepsi', 'her', 'herhangi', 'herkes', 'herkesin',
    'hiç', 'hiçbir', 'hiçbiri', 'hoş', 'hulasaten', 'iken', 'iki', 'ila', 'ile',
    'ilen', 'ilgili', 'ilk', 'illa', 'illaki', 'imdi', 'indinde', 'inen', 'insermi',
    'ise', 'ister', 'itibaren', 'itibariyle', 'itibarıyla', 'iyi', 'iyice', 'iyicene',
    'için', 'iş', 'işte', 'kadar', 'kaffesi', 'kah', 'kala', 'kannımca',
    'karşın', 'katrilyon', 'kaynak', 'kaçı', 'kelli', 'kendi', 'kendilerine',
    'kendini', 'kendisi', 'kendisine', 'kendisini', 'kere', 'kez', 'keza',
    'kezalik', 'keşke', 'keţke', 'ki', 'kim', 'kimden', 'kime', 'kimi', 'kimisi',
    'kimse', 'kimsecik', 'kimsecikler', 'külliyen', 'kırk',
    'kısaca', 'lakin', 'leh', 'lütfen', 'maada', 'madem', 'mademki', 'mamafih',
    'mebni',  'meğer', 'meğerki', 'meğerse', 'milyar', 'milyon', 'mu',
    'mü', 'mi', 'mı',  'nasıl', 'nasılsa', 'nazaran', 'naşi', 'ne', 'neden',
    'nedeniyle', 'nedenle', 'nedense', 'nerde', 'nerden', 'nerdeyse', 'nere',
    'nerede', 'nereden', 'neredeyse', 'neresi', 'nereye', 'netekim', 'neye', 'neyi',
    'neyse', 'nice', 'nihayet', 'nihayetinde', 'nitekim', 'niye', 'niçin', 'o',
    'olan', 'olarak', 'oldu', 'olduklarını', 'oldukça', 'olduğu', 'olduğunu',
    'olmadı', 'olmadığı', 'olmak', 'olması', 'olmayan', 'olmaz', 'olsa', 'olsun',
    'olup', 'olur', 'olursa', 'oluyor', 'on', 'ona', 'onca', 'onculayın', 'onda',
    'ondan', 'onlar', 'onlardan', 'onları', 'onların', 'onu',
    'onun', 'oracık', 'oracıkta', 'orada', 'oradan', 'oranca', 'oranla', 'oraya',
    'otuz', 'oysa', 'oysaki', 'pek', 'pekala', 'peki', 'pekçe', 'peyderpey', 'rağmen',
    'sadece', 'sahi', 'sahiden', 'sana', 'sanki', 'sekiz', 'seksen', 'sen', 'senden',
    'seni', 'senin', 'siz', 'sizden', 'sizi', 'sizin', 'sonra', 'sonradan',
    'sonraları', 'sonunda', 'tabii', 'tam', 'tamam', 'tamamen', 'tamamıyla',
    'tarafından', 'tek', 'trilyon', 'tüm', 'var', 'vardı', 'vasıtasıyla', 've',
    'velev', 'velhasıl', 'velhasılıkelam', 'veya', 'veyahut', 'ya', 'yahut',
    'yakinen', 'yakında', 'yakından', 'yakınlarda', 'yalnız', 'yalnızca', 'yani',
    'yapacak', 'yapmak', 'yaptı', 'yaptıkları', 'yaptığı', 'yaptığını', 'yapılan',
    'yapılması', 'yapıyor', 'yedi', 'yeniden', 'yenilerde', 'yerine',
    'yetmiş', 'yine', 'yirmi', 'yok', 'yoksa', 'yoluyla', 'yüz', 'yüzünden',
    'zarfında', 'zaten', 'zati', 'zira', 'çabuk', 'çabukça', 'çeşitli', 'çok',
    'çokları', 'çoklarınca', 'çokluk', 'çoklukla', 'çokça', 'çoğu', 'çoğun',
    'çoğunca', 'çoğunlukla', 'çünkü', 'öbür', 'öbürkü', 'öbürü', 'önce', 'önceden',
    'önceleri', 'öncelikle', 'öteki', 'ötekisi', 'öyle', 'öylece', 'öylelikle',
    'öylemesine', 'öz', 'üzere', 'üç',  'şayet', 'şey', 'şeyden', 'şeyi', 'şeyler',
    'şu', 'şuna', 'şuncacık', 'şunda', 'şundan', 'şunlar', 'şunları', 'şunu',
    'şunun', 'şura', 'şuracık', 'şuracıkta', 'şurası', 'şöyle',  'şimdi', 'şöyle'
}

from nltk.corpus import stopwords
turkish_stopwords = set(stopwords.words('turkish'))

# Tüm stopwords'leri birleştir
all_stopwords = turkish_stopwords.union(extended_turkish_stopwords)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Melek\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Melek\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Lemma ve Kök bulma fonksiyonları
def get_lemmas(word):
    """Bir kelimenin lemmalarını (köklerini) Zemberek ile bulur"""
    try:
        response = morphology_stub.AnalyzeWord(z_morphology.WordAnalysisRequest(input=word))
        if response.analyses:
            return response.analyses[0].lemmas
        return []
    except:
        return []

def get_stem(word):
    """Bir kelimenin kökünü Zemberek ile bulur"""
    lemmas = get_lemmas(word)
    return lemmas[0] if lemmas else word


In [3]:
# Çoklu kelimeler için toplu kök bulma (performans iyileştirmesi)
def get_stems_batch(words):
    """Kelime listesi için kök bulma işlemini toplu yapar"""
    results = {}
    for word in words:
        if word not in results:
            results[word] = get_stem(word)
    return results

In [5]:
def clean_srt_file(file_path):
    """SRT dosyasını temizler (zaman damgalarını ve numaraları kaldırır)"""
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    cleaned_lines = []
    current_line = ""
    
    for line in lines:
        line = line.strip()
        # Zaman damgası satırlarını ve numaralandırmayı atla
        if re.match(r'^\d+$', line) or '-->' in line:
            continue
        # Boş satırları atla
        if not line:
            if current_line:  # Eğer biriktirilen bir satır varsa ekle
                cleaned_lines.append(current_line)
                current_line = ""
            continue
        
        # Satırları birleştir (altyazılarda cümleler genelde birden fazla satıra bölünebilir)
        if current_line:
            current_line += " " + line
        else:
            current_line = line
    
    # Son satırı eklemeyi unutma
    if current_line:
        cleaned_lines.append(current_line)
        
    return cleaned_lines

In [6]:
from nltk import sent_tokenize


def split_into_sentences(lines):
    """Metni cümlelere böler"""
    sentences = []
    for line in lines:
        # NLTK'nın sent_tokenize fonksiyonunu kullanarak daha doğru cümle bölme
        line_sentences = sent_tokenize(line)
        for sentence in line_sentences:
            sentence = sentence.strip()
            if sentence:
                sentences.append(sentence)
    return sentences


In [7]:
def clean_sentence(sentence):
    """Cümleyi temizler"""
    # Altyazı notasyonlarını temizle ([gülüşmeler], (fısıldar) gibi)
    sentence = re.sub(r'\[.*?\]|\(.*?\)', '', sentence)
    
    # HTML etiketlerini kaldır
    sentence = re.sub(r'<.*?>', '', sentence)
    
    # Noktalama işaretlerini kaldır (sadece kelimeler, sayılar ve boşluklar kalsın)
    sentence = re.sub(r'[^\w\sğüşıöçĞÜŞİÖÇ0-9]', ' ', sentence)
    
    # Küçük harfe çevir (sayılar etkilenmez)
    sentence = sentence.lower()
    
    # Fazla boşlukları kaldır
    sentence = re.sub(r'\s+', ' ', sentence).strip()
    
    return sentence

In [8]:
def remove_stopwords_and_stem(sentence, stem_cache=None):
    """Stopwords kaldırır ve kök bulma işlemi yapar"""
    if stem_cache is None:
        stem_cache = {}
        
    words = word_tokenize(sentence)
    processed_words = []
    
    # Cache'de olmayan kelimeler için kök bulma işlemi yap
    new_words = [w for w in words if w not in stem_cache and w not in all_stopwords 
                 and (len(w) > 2 or w.isdigit() or re.match(r'^\w+-\d+\.?\d*$', w))]
    
    if new_words:
        new_stems = get_stems_batch(new_words)
        stem_cache.update(new_stems)
    
    # Cümledeki kelimeleri işle
    for word in words:
        if word not in all_stopwords and (len(word) > 2 or word.isdigit() or re.match(r'^\w+-\d+\.?\d*$', word)):
            if not word.isdigit() and not re.match(r'^\w+-\d+\.?\d*$', word):
                stemmed_word = stem_cache.get(word, get_stem(word))
                processed_words.append(stemmed_word)
            else:
                processed_words.append(word)
    
    return ' '.join(processed_words)

In [9]:
def process_all_files(directory):
    """Tüm dosyaları işler"""
    all_data = []
    stem_cache = {}  # Kök bulma için cache
    
    for file_path in directory.glob('*.srt'):
        print(f"İşleniyor: {file_path.name}")
        film_name = file_path.stem
        
        # 1. SRT dosyasını temizle
        cleaned_lines = clean_srt_file(file_path)
        
        # 2. Cümlelere böl
        sentences = split_into_sentences(cleaned_lines)
        
        # 3. Her cümleyi işle
        for sentence in sentences:
            # 4. Cümleyi temizle
            cleaned_sentence = clean_sentence(sentence)
            
            # 5. Stopwords kaldır ve kök bulma işlemi yap
            final_sentence = remove_stopwords_and_stem(cleaned_sentence, stem_cache)
            
            # Boş cümleleri atlama
            if final_sentence.strip():
                all_data.append({
                    'Film_Name': film_name,
                    'Original_Sentence': sentence,
                    'Processed_Sentence': final_sentence
                })

    return pd.DataFrame(all_data)

In [17]:
import pandas as pd
from pathlib import Path

# Film etiketleri sözlüğü
film_etiketleri = {
    "Titanic.srt": "romantik",
    "The Pianist.srt": "savaş",
    "Inception.srt": "bilim kurgu",
    "The Matrix.srt": "aksiyon",
    "The Shawshank Redemption.srt": "dram",
    "The Lord Of The Rings.srt": "fantastik",
    "Shutter Island.srt": "gerilim",
    "Leon.srt": "suç",
    "Saving Private Ryan.srt": "tarih",
    "Whiplash.srt": "müzik",
    "The Hangover.srt": "komedi",
    "The Conjuring.srt": "korku",
    "Frozen.srt": "animasyon",
    "Rocky.srt": "spor",
    "Blade Runner.srt": "distoptik",
    "Se7en.srt": "polisiye",
    "Pride & Prejudice.srt": "romantik",
    "Full Metal Jacket.srt": "savaş",
    "Interstellar.srt": "bilim kurgu",
    "Mad Max.srt": "aksiyon",
    "Requiem for a Dream.srt": "dram",
    "Stardust.srt": "fantastik",
    "Gone Girl.srt": "gerilim",
    "Goodfellas.srt": "suç",
    "The King's Speech.srt": "tarih",
    "Bohemian Rhapsody.srt": "müzik",
    "Superbad.srt": "komedi",
    "The Exorcist.srt": "korku",
    "Inside Out.srt": "animasyon",
    "Rush.srt": "spor",
    "Children of Men.srt": "distoptik",
    "The Silence of the Lambs.srt": "polisiye",
    "La La Land.srt": "romantik",
    "Harry Potter and the Sorcerer's Stone.srt": "fantastik",
    "Die Hard.srt": "aksiyon",
    "Forrest Gump.srt": "dram",
    "Prisoners.srt": "gerilim",
    "The Godfather.srt": "suç",
    "Schindler's List.srt": "tarih",
    "A Star is Born.srt": "müzik",
    "Anchorman.srt": "komedi",
    "Hereditary.srt": "korku",
    "Toy Story.srt": "animasyon",
    "The Fighter.srt": "spor",
    "The Road.srt": "distoptik",
    "Zodiac.srt": "polisiye"
}


In [18]:
def add_labels_to_dataframe(df, label_dict):
    """DataFrame'e etiketleri ekler"""
    df['Etiket'] = df['Film_Name'].apply(lambda x: label_dict.get(x + '.srt', 'bilinmiyor'))
    return df


In [19]:
from collections import Counter
def analyze_term_frequencies(df):
    """Her film türü için en sık kullanılan kelimeleri analiz eder"""
    genre_keywords = {}
    
    for genre in df['Etiket'].unique():
        # Bu türdeki tüm cümleleri al
        genre_sentences = df[df['Etiket'] == genre]['Processed_Sentence'].str.cat(sep=' ')
        # Kelimelere böl
        words = word_tokenize(genre_sentences)
        # En sık kullanılan 20 kelimeyi bul
        word_counts = Counter(words).most_common(20)
        genre_keywords[genre] = word_counts
    
    return genre_keywords

In [20]:
def main():
    # Altyazı dosyalarının bulunduğu klasör
    dosya_klasoru = Path(r"C:\Users\Melek\yapayZeka\FilmTemaAnaliziProje\Filmler")
    
    # 1. Tüm dosyaları işle
    print("Dosyalar işleniyor...")
    final_df = process_all_files(dosya_klasoru)
    
    # 2. Etiketleri ekle
    print("Etiketler ekleniyor...")
    labeled_df = add_labels_to_dataframe(final_df, film_etiketleri)
    
    # 3. Kelime sıklığı analizi yap
    print("Kelime sıklığı analizi yapılıyor...")
    genre_keywords = analyze_term_frequencies(labeled_df)
    
    # Sonuçları kaydet
    output_dir = Path(r"C:\Users\Melek\yapayZeka\FilmTemaAnaliziProje\İşlenmişVeriler4")
    output_dir.mkdir(parents=True, exist_ok=True)  # Klasör yoksa oluştur
    
    # Tüm verileri tek CSV'de kaydet - Sadece işlenmiş cümle ve etiket ile
    all_data_path = output_dir / "tum_filmler_etiketli_stemmed.csv"
    labeled_df[['Film_Name', 'Processed_Sentence', 'Etiket']].to_csv(all_data_path, index=False, encoding='utf-8-sig')
    print(f"Tüm veriler kaydedildi: {all_data_path}")
    
    # Filmler ayrı ayrı CSV'ler halinde kaydet - Sadece işlenmiş cümle ve etiket ile
    for film_adi, grup in labeled_df.groupby('Film_Name'):
        # Dosya adındaki geçersiz karakterleri temizle
        safe_name = re.sub(r'[\\/*?:"<>|]', "_", film_adi)
        file_path = output_dir / f"{safe_name}_etiketli_stemmed.csv"
        
        # Yalnızca işlenmiş cümle ve etiket sütunları
        grup[['Processed_Sentence', 'Etiket']].to_csv(file_path, index=False, encoding='utf-8-sig')
        print(f"Kaydedildi: {file_path}")
    
    # Türlere göre sık kelime analizi sonuçlarını kaydet
    with open(output_dir / "tur_kelime_analizi.txt", "w", encoding="utf-8") as f:
        for genre, keywords in genre_keywords.items():
            f.write(f"Film Türü: {genre}\n")
            for word, count in keywords:
                f.write(f"  {word}: {count}\n")
            f.write("\n")
    
    print("İşlem tamamlandı!")

if __name__ == "__main__":
    main()

Dosyalar işleniyor...
İşleniyor: A Star is Born.srt
İşleniyor: Anchorman.srt
İşleniyor: Blade Runner.srt
İşleniyor: Bohemian Rhapsody.srt
İşleniyor: Children of Men.srt
İşleniyor: Die Hard.srt
İşleniyor: Forrest Gump.srt
İşleniyor: Frozen.srt
İşleniyor: Full Metal Jacket.srt
İşleniyor: Gone Girl.srt
İşleniyor: Goodfellas.srt
İşleniyor: Harry Potter and the Sorcerer's Stone.srt
İşleniyor: Hereditary.srt
İşleniyor: Inception.srt
İşleniyor: Inside Out.srt
İşleniyor: Interstellar.srt
İşleniyor: La La Land.srt
İşleniyor: Leon.srt
İşleniyor: Mad Max.srt
İşleniyor: Pride & Prejudice.srt
İşleniyor: Prisoners.srt
İşleniyor: Requiem for a Dream.srt
İşleniyor: Rocky.srt
İşleniyor: Rush.srt
İşleniyor: Saving Private Ryan.srt
İşleniyor: Schindler's List.srt
İşleniyor: Se7en.srt
İşleniyor: Shutter Island.srt
İşleniyor: Stardust.srt
İşleniyor: Superbad.srt
İşleniyor: The Conjuring.srt
İşleniyor: The Exorcist.srt
İşleniyor: The Fighter.srt
İşleniyor: The Godfather.srt
İşleniyor: The Hangover.srt
İşlen